In [3]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

In [1]:
import numpy as np
import os

function_dirs = ["function_1", "function_2", "function_3", "function_4", 
                 "function_5", "function_6", "function_7", "function_8"]

new_queries = [
    np.array([0.716333, 0.391319]), np.array([0.589302, 0.216414]), np.array([0.453012, 0.249551, 0.562613]), np.array([0.500265, 0.440834, 0.326775, 0.367054]), np.array([0.084258, 0.166152, 0.967563, 0.33688 ]), np.array([0.766373, 0.350032, 0.825146, 0.834555, 0.046373]), np.array([0.067914, 0.265016, 0.308222, 0.215279, 0.360383, 0.647403]), np.array([0.18634 , 0.142909, 0.142472, 0.252725, 0.430269, 0.701337,
       0.362055, 0.490818])
]

new_observations = [6.469674680863722e-45, 0.1899188843838252, -0.05358080540579265, -1.3792129733736043, 120.70231913989132, -0.6948883166320663, 2.5371557473132045, 9.8114186378301]

def update_function_files(function_dir, new_query, new_observation):
    input_path = os.path.join(function_dir, "initial_inputs.npy")
    output_path = os.path.join(function_dir, "initial_outputs.npy")
    
    inputs = np.load(input_path)
    outputs = np.load(output_path)
    
    updated_inputs = np.vstack([inputs, new_query])
    updated_outputs = np.append(outputs, new_observation)
    
    np.save(input_path, updated_inputs)
    np.save(output_path, updated_outputs)

for i, function_dir in enumerate(function_dirs):
    update_function_files(function_dir, new_queries[i], new_observations[i])

print("All files have been updated successfully.")

All files have been updated successfully.


In [3]:
import numpy as np
import os

function_dirs = ["function_1", "function_2", "function_3", "function_4", 
                 "function_5", "function_6", "function_7", "function_8"]

def check_function_files(function_dir):
    input_path = os.path.join(function_dir, "initial_inputs.npy")
    output_path = os.path.join(function_dir, "initial_outputs.npy")
    
    inputs = np.load(input_path)
    outputs = np.load(output_path)
    
    print(f"--- {function_dir} ---")
    print("Inputs:")
    print(inputs)
    print("\nOutputs:")
    print(outputs)
    print("\n" + "="*40 + "\n")

for function_dir in function_dirs:
    check_function_files(function_dir)

--- function_1 ---
Inputs:
[[0.31940389 0.76295937]
 [0.57432921 0.8798981 ]
 [0.73102363 0.73299988]
 [0.84035342 0.26473161]
 [0.65011406 0.68152635]
 [0.41043714 0.1475543 ]
 [0.31269116 0.07872278]
 [0.68341817 0.86105746]
 [0.08250725 0.40348751]
 [0.88388983 0.58225397]
 [0.4        0.4       ]
 [0.42       0.42      ]
 [0.424242   0.424242  ]
 [0.512075   0.620688  ]
 [0.512075   0.620688  ]
 [0.512075   0.620688  ]
 [0.38       0.4       ]
 [0.716333   0.391319  ]]

Outputs:
[ 1.32267704e-079  1.03307824e-046  7.71087511e-016  3.34177101e-124
 -3.60606264e-003 -2.15924904e-054 -2.08909327e-091  2.53500115e-040
  3.60677119e-081  6.22985647e-048  9.78702809e-002  9.89816016e-001
  9.63511895e-001  9.63511895e-001 -3.48082708e-011 -3.48082708e-011
 -1.74937062e-003  6.46967468e-045]


--- function_2 ---
Inputs:
[[0.66579958 0.12396913]
 [0.87779099 0.7786275 ]
 [0.14269907 0.34900513]
 [0.84527543 0.71112027]
 [0.45464714 0.29045518]
 [0.57771284 0.77197318]
 [0.43816606 0.685018

In [23]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm

def generate_new_query(function_dir, exploration_exploitation_tradeoff):
    input_path = os.path.join(function_dir, "initial_inputs.npy")
    output_path = os.path.join(function_dir, "initial_outputs.npy")
    
    inputs = np.load(input_path)
    outputs = np.load(output_path)
    
    kernel = Matern()
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6)
    
    gp.fit(inputs, outputs)
    
    def acquisition_function(x, xi):
        mu, sigma = gp.predict(x.reshape(1, -1), return_std=True)
        
        f_max = np.max(outputs)
        
        Z = (mu - f_max - xi) / sigma
        EI = (mu - f_max - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
        
        return EI

    dim = inputs.shape[1]
    candidates = np.random.rand(5000, dim)

    xi = (100 - exploration_exploitation_tradeoff) / 100
    acquisition_values = np.array([acquisition_function(x, xi) for x in candidates])

    best_candidate = candidates[np.argmax(acquisition_values)]
    
    banana_format = "-".join([f"0.{str(round(val, 6)).split('.')[1]}" for val in best_candidate])
    
    return banana_format

function_dir = "function_8"
exploration_exploitation_tradeoff = 5000
new_query_banana = generate_new_query(function_dir, exploration_exploitation_tradeoff)

print("New Query in Banana format:", new_query_banana)


New Query in Banana format: 0.132081-0.347656-0.175477-0.153514-0.628277-0.467261-0.310183-0.558869
